In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import time 
import googleapiclient.discovery as build
from googleapiclient.errors import HttpError

In [3]:
# Load the YOUTUBE API key from the .env file
load_dotenv()
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")
if YOUTUBE_API_KEY: 
    print("API key loaded successfully!")
else:
    print("Error: API Key not found. Check your .env file.")

api_service_name = "youtube"
api_version = "v3"

# Initialize the YouTube API client
youtube = build.build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
youtube

API key loaded successfully!


In [4]:
video_id = 'UDVOuh-LDx8' 

video_response = youtube.videos().list(
    part = "id,snippet,contentDetails,statistics",
    id = video_id, # Replace with the actual video ID
).execute()   

if not video_response:
    print("No response from the API.")

print(video_response)
print(video_response['items'][0].keys())

{'kind': 'youtube#videoListResponse', 'etag': 'ZBKNGl5WNl7U0az9xPQAhMvkReg', 'items': [{'kind': 'youtube#video', 'etag': 'q7l1VYYpz3hTe47YsKLgcNH5S1c', 'id': 'UDVOuh-LDx8', 'snippet': {'publishedAt': '2025-03-14T17:00:36Z', 'channelId': 'UC-lHJZR3Gqxm24_Vd_AJ5Yw', 'title': 'We finally decided.', 'description': "Get exclusive NordVPN deal here ➵ https://NordVPN.com/pewdiepie It’s risk free with Nord’s 30 day money-back guarantee!✌\n\n🌎 Get an exclusive 15% discount on Saily data plans! Use code pewdiepie at checkout. Download Saily app or go to https://saily.com/pewdiepie ⛵\n\nOfficial ƿ૯ωძɿ૯ƿɿ૯ TikTok: https://www.tiktok.com/@pewdiepie\n\n∞ Official ƿ૯ωძɿ૯ƿɿ૯ Infinity Stream: https://www.twitch.tv/pewdiepie\n\n⚙️ My Setup (affiliate links)⚙️\n🖥️  PC: https://howl.me/clHcdRoIePi\n🎧 Headphones: https://www.anrdoezrs.net/click-100814745-14330447\n🪑 Chair: https://tinyurl.com/clutchpdp\n⌨️ 🖱️ Keyboard & Mouse: https://tinyurl.com/ghostpdp\n\n👟 Sneakers: https://howl.me/clHccGkb0M4\n\n🕹️ Pe

In [4]:
# Useful links: https://medium.com/mcd-unison/youtube-data-api-v3-in-python-tutorial-with-examples-e829a25d2ebd#a60a
# https://developers.google.com/youtube/v3/docs/search

video_data = video_response['items'][0]

video_snippet = video_data['snippet']
video_statistics = video_data['statistics']
video_content_details = video_data['contentDetails']

# Crreate a dictionrary for the data of the video
video_info = {
    'Channel_name': video_snippet['channelTitle'],
    'video_id': video_data['id'],
    'video_url': f"https://www.youtube.com/watch?v={video_data['id']}",
    'video_title': video_data['snippet']['title'],
    'video_publication_date': video_data['snippet']['publishedAt'],
    'video_duration': video_content_details['duration'],
    'video_view_count': int(video_statistics['viewCount']),
    'video_like_count': int(video_statistics['likeCount']),
    'video_comment_count': int(video_statistics['commentCount'])
}

video_info


{'Channel_name': 'PewDiePie',
 'video_id': 'UDVOuh-LDx8',
 'video_url': 'https://www.youtube.com/watch?v=UDVOuh-LDx8',
 'video_title': 'We finally decided.',
 'video_publication_date': '2025-03-14T17:00:36Z',
 'video_duration': 'PT29M9S',
 'video_view_count': 3573476,
 'video_like_count': 202492,
 'video_comment_count': 7886}

In [ ]:
'''
next_page_token = None
next_page_token = response.get('nextPageToken')
response = youtube.commentThreads().list(
    part='snippet,replies',            
    videoId=video_id,             
    maxResults=10,                    
    textFormat='plainText',
    pageToken=next_page_token,
).execute()

# print(response)
print(response['items'][0]['id'])
print(response['items'][0].keys())
# print(response['items'][0]['snippet']['topLevelComment']['snippet']['textDisplay'])

next_page_token = None
all_comments = []
# Include also the time needed to save the comments
try:
    start_time = time.time()
    for i in range(min(100, video_info['video_comment_count'])):
        response = youtube.commentThreads().list(
            part='snippet,replies',            
            videoId=video_id,             
            maxResults=1,                    
            textFormat='plainText',
            pageToken=next_page_token,
        ).execute()
        next_page_token = response.get('nextPageToken')
        # print(response['items'][0]['snippet']['topLevelComment']['snippet']['textDisplay'])
        comment = response['items'][0]['snippet']['topLevelComment']['snippet']['textDisplay']
        all_comments.append(comment)
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Time taken to fetch comments: {round(total_time,2)} seconds")
    print(f"The total number of comments is {len(all_comments)}")
except HttpError as e:
        print(f"An HTTP error {e.resp.status} occurred:\n{e.content}")

'''


In [6]:
# Link for the docummentaion of the youtube.commentThreads().list() method:
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/python/latest/youtube_v3.commentThreads.html

all_comments = []
next_page_token = None
max_numner_of_comments = video_info['video_comment_count']
# max_numner_of_comments = 1000
try:
    start_time = time.time()
    while len(all_comments) < max_numner_of_comments:
        response = youtube.commentThreads().list(
            part='snippet,replies',            
            videoId=video_id,             
            maxResults=min(100, max_numner_of_comments - len(all_comments)),                    
            textFormat='plainText',
            pageToken=next_page_token,
        ).execute()
        # If no comment threads are found, break the loop
        if not response.get('items'):
            print("No comment threads found.")
            break

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']

            comment_info = {
                'comment_id': item['id'],
                'author': comment['authorDisplayName'],
                'author_id': comment.get('authorChannelId', {}).get('value', ''),
                'text': comment['textDisplay'],
                'like_count': comment['likeCount'],
                'published_at': comment['publishedAt'],
                'updated_at': comment['updatedAt'],
            }

            all_comments.append(comment_info)

            if len(all_comments) >= max_numner_of_comments:
                print("Reached the maximum number of comments.")
                break

            # Check if there are more comments to fetch
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
            
            # Sleep to avoid hitting rate limits
            # time.sleep(0.5)
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Time taken to fetch comments: {round(total_time,2)} seconds")
    print(f"The total number of comments is {len(all_comments)}")
except HttpError as e:
    print(f"An HTTP error {e.resp.status} occurred:\n{e.content}")

Reached the maximum number of comments.
Time taken to fetch comments: 24.88 seconds
The total number of comments is 7886


In [23]:
# Now we move on to save the data first as a pandas dataframe and then in a csv file 
comments_df = pd.DataFrame(all_comments)

# Save the comments to a CSV file
folder_path = '\\youtube-sentiment-analysis\\data'
video_info['video_title']

# Create the CSV file path using the video title
csv_file_path = os.path.join(folder_path, f"{video_info['Channel_name']}_{video_info['video_title']}_comments.csv")

# Save the video comments to a CSV file
comments_df.to_csv(csv_file_path, index=False)

print(f'Video comments with title: {video_info["video_title"]} saved to CSV file: {csv_file_path}')


Video comments with title: We finally decided. saved to CSV file: \youtube-sentiment-analysis\data\PewDiePie_We finally decided._comments.csv
